In [48]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

class Free_Agency:
    def __init__(self, link):
        self.link = link
        self.request = requests.get(self.link)
        self.soup = BeautifulSoup(self.request.text, 'html.parser')
        self.results = self.soup.find_all('tr')[1:]
    def Create_Table(self):
        Player = []
        Stats = []
        Terms = []
        for result in self.results:
            Player += [result.contents[1].text] if (result.contents[1].text != 'Rk') else []
            try:
                Stats += [result.contents[6].text]
            except:
                pass
            Terms += [result.contents[9].text] if (result.contents[9].text != 'Type') else []
        Data = pd.DataFrame({'Player' : Player, 'Stats' : Stats, 'Terms' : Terms})
        Year = int(self.link[-4:])
        Data['Year'] = Year
        return(Data)
    
a = 'https://www.basketball-reference.com/friv/free_agents.cgi?year=2017'
c = []
for i in range(2018, 2020):
    i = str(i)
    c += [a.replace('2017', i)]

table = Free_Agency('https://www.basketball-reference.com/friv/free_agents.cgi?year=2017').Create_Table()
for i, k in enumerate(c):
    table = table.append(Free_Agency(k).Create_Table())

In [49]:
table.drop(table[table['Stats'] == 'Did not play'].index, inplace = True)
table.drop(table[table['Terms'] == ''].index, inplace = True)
table.drop(table[table['Terms'].str[:23] == 'Will reportedly play in'].index, inplace = True)

In [50]:
table.to_csv('Basketball-Reference Free Agency.csv')

In [51]:
table.shape

(289, 4)